# [TorchVision 객체 검출 미세조정](https://tutorials.pytorch.kr/intermediate/torchvision_tutorial.html)

## data set 정의
- 참조 스크립트를 통해 새로운 사용자 정의 데이터 셋 추가를 쉽게 진행할 수 있음
    - 표쥰 `torch.utils.data.Dataset` 클래스를 상속 받고 `__len__`과 `__getitem__` 메서드 구현
    - 이미지 : PIL 이미지의 크기 (H, W)
    - 대상
        - boxes(FloatTensor[N, 4]): N 개의 바운딩 박스 좌표 [x0, y0, x1, y1]
            - 0 < x < W, 0 < y < H
        - labels(Int64Tensor[N]): 바운딩 박스 마다의 라벨 정보
        - image_id(Int64Tensor[1]): 이미지 구분자
        - area(Tensor[N]): 바운딩 박스의 면적
            - 박스간의 점수를 내기위한(작음, 중간, 큰)COCO평가를 기준으로 함
        - iscrowd(UInt8Tensor[N]): 이 값이 참일경우 평가에서 제외
        - (선택적)masks(UInt8Tensor[N, H, W]): N개의 객체 마다의 분할 마스크 정보
        - (선택적)keypoints(FloatTensor[N, K, 3]): N개의 객체마다의 키포인트 정보
            - 키포인트는 [x, y, visibility] 형태의 값
                - 0이면 키포인트는 보이지 않음을 의미
                - 데이터 증강의 경우 키포인트 좌우 반전의 개념은 데이터 표엔에 따라 달라짐
        

- 평가 스크립트는 `pypcocotools`를 이용
- 학습 중에 가로 세로 비율 그룹화를 사용하는 경우 메소드를 구현
    - `get_height_and_width`

## PennFudan을 위한 사용자 정의 데이터셋 작성
- __getitem__() 부분에서 mask와 obj_ids부분에서 각 인스턴스들에 대해 boolean indexing하는 부분이 있는데
    - 여기서 비교되는 각 shapeㅇ

- [Pedestrain data](https://www.cis.upenn.edu/~jshi/ped_html/)

In [19]:
import os
import numpy as np
import torch
from PIL import Image

class PennFudanDataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # 모든 이미지 파일들을 읽고 정렬
        # 이미지와 분할 마스크 정렬 확인
        self.imgs = list(sorted(os.listdir(os.path.join(root, 'PNGImages'))))
        self.masks = list(sorted(os.listdir(os.path.join(root, 'PedMasks'))))
        
    def __getitem__(self, idx):
        # 이미지와 마스크를 읽어옴
        img_path = os.path.join(self.root, 'PNGImages', self.imgs[idx])
        mask_path = os.path.join(self.root, 'PedMasks', self.masks[idx])
        # RGB로 변환해줌
        # shape : [2, ] value : [536, 559]
        img = Image.open(img_path).convert('RGB')
        # mask는 2차원 값으로 색상 채널이 없음
        mask = Image.open(mask_path)
        # numpy로 변환
        mask = np.array(mask)
        # 인스턴스들은 다른 색깔로 인코딩 되어있음
        # 중복되는 요소 지우는데 
        # 0 번째는 배경이므로 1: slicing
        obj_ids = np.unique(mask)
        # shape : [2, ] value : [1,2]
        obj_ids = obj_ids[1:]
        
        # 컬러 인코딩된 마스크를 바이너리 마스크 세트로 나눔
        # [:, None, None] 이런식으로 하면 None 부분에대한 차원이 생김
        # shape : [2, 1, 1]
        # mask 와 obj_idx의 shape은 다른데 어떤식으로든 가장 큰 shape에 맞춰서 boolean indexing 값 볼 수 있음
        # shape : [2, 536, 559]
        masks = mask == obj_ids[:, None, None]
        
        # 각 마스크의 바운딩 박스좌표를 얻음
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # 모든 것을 torch.Tensor 타입으로 변환합니다
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # 객체 종류는 한 종류만 존재합니다(역자주: 예제에서는 사람만이 대상입니다)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # 모든 인스턴스는 군중(crowd) 상태가 아님을 가정합니다
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

## 모델 정의하기
- Faster R-CNN에 기반한 Mask R-CNN
- TorchVision 모델주(미리 학습된 모델들을 모아놓은 공간)에서 사용 가능한 모델들 중 하나를 이용해 모델을 수정하려면 두 가지 상황이 있음
    - 1. 미리 학습된 모델에서 시작, 마지막 레이어만 파인튜닝
    - 2. 모델의 백본을 다른 백본으로 교체
        - ResNet -> Mobilenet 같은

### 미리 학습된 모델에서 시작, 마지막 레이어만 파인튜닝
- COCO로 미리 학습된 모델읽고 분류기 교체

In [20]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# COCO로 미리 학습된 모델 읽기
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# 분류기를 새로운 것으로 교체, num_classes는 사용자가 정의
num_classes = 2 # 1 클래스(사람) + 배경
# 분류기에서 사용할 입력 특징의 차원 정보를 얻음
in_features = model.roi_heads.box_predictor.cls_score.in_features
# 미리 학습된 모델의 머리 부분을 새로운 것으로 교체
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

### 다른 백본을 추가하도록 모델을 수정
- backbone : ResNet101 -> mobilenet_v2, output_channels = 1280
- num_classes : 2
- rpn_anchor_generator : 5개의 서로다른 크기와 3개의 다른 측면 비율 5x3개의 anchor
- box_roi_pool : 관심영역 자르기 및 재할당 후 자르기 크기 수행하는데 사용할 피쳐 맵 정의
- 위의 모든 값을 FasterRCNN의 인자로 주어 합침

In [21]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# 분류 목적으로 미리 학습된 모델을 로드하고 특징들만을 리턴
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# Faster RCNN은 백본의 출력 채널 수를 알아야함
# mobilenetv2의 경우 1280이므로 여기에 추가
backbone.out_channels = 1280

# RPN(Region Proposal Network)이 5개의 서로 다른 크기와 3개의 다른 측면 비율을 가진
# 5 x 3개의 앵커를 공간 위치마다 생성하도록 함
# 각 특징 맵이 잠재적으로 다른 사이즈와 측면 비율을 가질 수 있기 때문에 Tuple[Tuple[int]] 타입을 가지도록 함
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))
# 관심 영역의 자르기 및 재할당 후 자르기 크기를 수행하는데 사용할 피쳐 맵을 정의
# 만약 백본이 텐서를 리턴할 때, feaumap_names는 [0]이 될 것이라 예상
# 일반적으로 백본은 OrderedDict[Tensor] 타입을 리턴해야 함
# 그리고 특징맵에서 사용할 feaumap_names 값을 정할 수 있음
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
                                                output_size=7,
                                                sampling_ratio=2)

# 조각들을 Faster RCNN 모델로 합침
model = FasterRCNN(backbone,
                   num_classes=2,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)

In [22]:
print(model)

FasterRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm

## PennFudan 데이터셋을 위한 인스턴스 분할 모델
- 데이터 셋이 적기때문에 미리학습된 모델에서 파인튜닝 하는식으로 진행

In [23]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # COCO 에서 미리 학습된 인스턴스 분할 모델을 읽어옵니다
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # 분류를 위한 입력 특징 차원을 얻습니다
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # 미리 학습된 헤더를 새로운 것으로 바꿉니다
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # 마스크 분류기를 위한 입력 특징들의 차원을 얻습니다
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # 마스크 예측기를 새로운 것으로 바꿉니다
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

## 하나로 합치기

In [24]:
import torchvision.transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        # 학습시 50% 확률로 영상 좌우 반전
        transforms.append(T.RandomHorizontalFlip(0.5))
    # compopse 통해 ToTensor나 , normalization 등 crop이ㄹ등등 
    return T.Compose(transforms)

- git clone https://github.com/pytorch/vision.git
- pip install Cython
- pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI
- vision/references/detection/engine, utils, transforms, coco_eval, coco_utils를 해당결로로 옮겨줌

In [25]:
from engine import train_one_epoch, evaluate
import utils

def main():
    # 학습을 GPU로 진행하되 GPU가 가용하지 않으면 CPU로 합니다
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # 우리 데이터셋은 두 개의 클래스만 가집니다 - 배경과 사람
    num_classes = 2
    # 데이터셋과 정의된 변환들을 사용합니다
    dataset = PennFudanDataset('./data/PennFudanPed', get_transform(train=True))
    dataset_test = PennFudanDataset('./data/PennFudanPed', get_transform(train=False))

    # 데이터셋을 학습용과 테스트용으로 나눕니다(역자주: 여기서는 전체의 50개를 테스트에, 나머지를 학습에 사용합니다)
    indices = torch.randperm(len(dataset)).tolist()
    dataset = torch.utils.data.Subset(dataset, indices[:-50])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

    # 데이터 로더를 학습용과 검증용으로 정의합니다
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=0,
        collate_fn=utils.collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=0,
        collate_fn=utils.collate_fn)

    # 도움 함수를 이용해 모델을 가져옵니다
    model = get_model_instance_segmentation(num_classes)

    # 모델을 GPU나 CPU로 옮깁니다
    model.to(device)

    # 옵티마이저(Optimizer)를 만듭니다
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # 학습률 스케쥴러를 만듭니다
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # 10 에포크만큼 학습해봅시다
    num_epochs = 10

    for epoch in range(num_epochs):
        # 1 에포크동안 학습하고, 10회 마다 출력합니다
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # 학습률을 업데이트 합니다
        lr_scheduler.step()
        # 테스트 데이터셋에서 평가를 합니다
        evaluate(model, data_loader_test, device=device)

    print("That's it!")

In [26]:
main()

TypeError: __call__() takes 2 positional arguments but 3 were given